In [31]:
# This is a base location recommender, look at location_recommended_with_friends_network
# 
import pandas
import numpy
from scipy import sparse

# user-item matrix generated in earlier file
user_item_sparse = sparse.load_npz('generated_data/user_item_matrix.npz')
user_item_csr = user_item_sparse.tocsr()

user_loc_f = numpy.load('generated_data/user_item_matrix_columns.npz')
user_list = list(user_loc_f['arr_0'])
loc_list = list(user_loc_f['arr_1'])

P_Q_f = numpy.load('generated_data/user_item_decomposed.npz')
P = P_Q_f['arr_0']
Q = P_Q_f['arr_1']


In [27]:
def recommend_locations_base(userid):
    usern = user_list.index(userid)
    
    predictions = numpy.dot(P[usern,:][numpy.newaxis],Q.T) # we get the predictions by dot product of P.Q [userid]
    predictions = predictions[0] #drop the two-dimensions
    predictions_idx = [x for _,x in sorted(zip(predictions,range(len(predictions))),reverse=True)]

    recommendations = numpy.array(loc_list)[predictions_idx]
    return recommendations
       
locations = recommend_locations_base(3)
print(locations)

[420315  21714   9241 ...  17144  13791  13776]


In [28]:
def recommend_locations(userid,K=10):
    recommendations = recommend_locations_base(userid)
    
    # filter the recommended locations if its already visited
    filtered = []
    n = 0
    usern = user_list.index(userid)
    while len(filtered) < K and n < len(recommendations):
        locn = loc_list.index(recommendations[n])
        if user_item_csr[usern,locn] == 0:
            filtered.append(recommendations[n])
        n += 1
    
    return filtered

In [29]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

userw=widgets.SelectionSlider(
    options=user_list,
    value=3,
    description='UserId:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True
)
#display(w)

def printer(userid):
    print("Recommended Locations:")
    print(recommend_locations(userid))
    
interact(printer,userid=userw)
print("")

interactive(children=(SelectionSlider(continuous_update=False, description='UserId:', index=2, options=(1, 2, …